Je vais analyser seulement la ville de Bordeaux parce qu'il y a un gros volume de vente. 

In [ ]:
# IPython extension reloading modules before user enters code.
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
%matplotlib inline

from functions import split

In [ ]:
df0 = pd.read_csv('./data/RAW/valeursfoncieres-2019.txt', decimal = ',', sep = '|')

In [ ]:
df0.columns.tolist()

In [ ]:
df0 = df0.drop(columns = ['Code service CH',
 'Reference document',
 '1 Articles CGI',
 '2 Articles CGI',
 '3 Articles CGI',
 '4 Articles CGI',
 '5 Articles CGI',
 'No disposition',
 'Date mutation',
 'No voie',
 'B/T/Q',
 'Type de voie',
 'Code voie',
 'Voie',
 'Code departement',
 'Code commune',
 'Prefixe de section',
 'Section',
 'No plan',
 'No Volume',
 '1er lot',
 'Surface Carrez du 1er lot',
 '2eme lot',
 'Surface Carrez du 2eme lot',
 '3eme lot',
 'Surface Carrez du 3eme lot',
 '4eme lot',
 'Surface Carrez du 4eme lot',
 '5eme lot',
 'Surface Carrez du 5eme lot',
 'Nombre de lots',
 'Code type local',
 'Identifiant local',
 'Nature culture',
 'Nature culture speciale'])

In [ ]:
df = df0.copy()   

In [ ]:
def preprocessing(df):
    df = df.drop_duplicates()
    df = df.dropna(subset = ['Type local'])
    df = df[df['Type local'].isin(['Maison','Appartement'])]
    df = df[df['Commune'].isin(['BORDEAUX'])]
    df = df[df['Nature mutation'].isin(['Vente'])]
    df = df.dropna(subset = ['Code postal', 'Valeur fonciere'])
    df = df[df["Surface reelle bati"] > 8]
    df = df[df["Nombre pieces principales"] >= 1]
    df = df.fillna(0)

    return df

In [ ]:
df  = preprocessing(df)

In [ ]:
df = df.drop(columns = ['Nature mutation', 'Commune'])

In [ ]:
df = df.drop(columns = ['Type local'])

In [ ]:
df = df.to_csv('./data/CURATED/csv_clean_bordeaux.csv')

In [ ]:
df_bordeaux = pd.read_csv('./data/CURATED/csv_clean_bordeaux.csv', sep = ',', decimal = ',', usecols = ['Unnamed: 0', 'Valeur fonciere', 'Code postal', 'Surface reelle bati', 'Nombre pieces principales', 'Surface terrain'])
df_bordeaux_anomaly = pd.read_csv('./data/CURATED/anomaly_csv_clean_bordeaux.csv', sep = ',', decimal = ',')

In [ ]:
df_bordeaux_anomaly = df_bordeaux_anomaly.drop(columns = ['Unnamed: 0', 'Valeur fonciere', 'Code postal', 'Nombre pieces principales', 'Surface terrain', 'Surface reelle bati', 'scores'])

In [ ]:
df_bordeaux_anomaly = df_bordeaux_anomaly.rename(columns = {'Unnamed: 0.1':'Unnamed: 0'})

In [ ]:
df_bordeaux = df_bordeaux.merge(df_bordeaux_anomaly, how = 'left', on = 'Unnamed: 0')

In [ ]:
df_bordeaux = df_bordeaux.fillna(0)
df_bordeaux = df_bordeaux[df_bordeaux["anomaly"] == 0]
df_bordeaux = df_bordeaux.drop(columns = ['Unnamed: 0', 'anomaly'])

In [ ]:
df_bordeaux['Valeur fonciere']  = df_bordeaux['Valeur fonciere'].astype(int)

In [ ]:
df_bordeaux = df_bordeaux[df_bordeaux['Valeur fonciere'].between(df_bordeaux['Valeur fonciere'].quantile(.10), df_bordeaux['Valeur fonciere'].quantile(.90))]

In [ ]:
df_bordeaux = df_bordeaux.to_csv('./data/CURATED/csv_clean_bordeaux_2.csv')

In [ ]:
df_clean = pd.read_csv('./data/CURATED/csv_clean_bordeaux_2.csv', usecols=['Valeur fonciere', 'Code postal', 'Surface reelle bati'])
df_clean['Prix moyen m²'] = df_clean['Valeur fonciere']/df_clean['Surface reelle bati']
df_prix_moyen = df_clean.groupby('Code postal').mean('Prix moyen m²').drop(columns = ['Valeur fonciere', 'Surface reelle bati'])

dict_prix_moyen = df_prix_moyen.to_dict()